In [1]:
import nltk
import pandas as pd 
import numpy as np
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings('ignore')
import math
from sklearn.model_selection import train_test_split 


In [2]:
data=pd.read_csv("/Users/a0s0iro/PycharmProjects/AI-DSE_MTECH/NLP/NER_Dataset.csv")

In [3]:
#### POS tag counter

In [4]:
## Length of POS Tag and Word should be same for a valid tagging
def checkValidaTagging(WordColumn,TagColumn):
    wordlist=WordColumn[1:-1].replace("'","").split(",")
    taglist=TagColumn[1:-1].replace("'","").split(",")
    return len(wordlist)==len(taglist)

def convertStringtoList(data,wordColumn=True):
    wordlist=data["Word"][1:-1].replace("'","").replace(" ","").split(",")
    taglist=data["POS"][1:-1].replace("'","").replace(" ","").split(",")
    specialchar='[@_!#$%^&*()<>?/\|}{~:],.;:`` '
    FinalwordList=[]
    FinalPosList=[]
    for i in range(len(wordlist)):
        if wordlist[i] not in specialchar and taglist[i] not in  specialchar:
            FinalPosList.append(taglist[i])
            FinalwordList.append(wordlist[i])
    if wordColumn:
        return FinalwordList
    else:
        return FinalPosList


## NUmber of words and unique words in each sentence 
def wordCount(data,uniqueWords=False):
    wordlist=data["WordList"]
    taglist=data["POSList"]
    if uniqueWords:
        return len(set(tuple(wordlist)))
    
    return len(wordlist)

In [5]:
def WordToTagDic(data):
    # wordlist=WordColumn[1:-1].replace("'","").replace(" ","").split(",")
    # taglist=TagColumn[1:-1].replace("'","").replace(" ","").split(",")
    # WordTagColumn
    wordlist=data["WordList"]
    taglist=data["POSList"]
    
    tagDic={}
    for i in range(len(wordlist)):
        word=wordlist[i]
        postag=taglist[i]


        if postag in tagDic:
            wordFreqDic=tagDic[postag]
            if word in wordFreqDic:
                wordFreqDic[word]+=1
            else:
                wordFreqDic[word]=1
        else:
            wordFreqDic={word:1}
            tagDic[postag]=dict(wordFreqDic)

    
    return tagDic
    

In [6]:


data["validTagging"]=data.apply(lambda x: checkValidaTagging(x[1],x[2]),axis=1)

notValidIndex=list(data[data["validTagging"]==False].index) 
data=data.drop(notValidIndex)

data["WordList"]=data.apply(lambda x:convertStringtoList(x),axis=1)
data["POSList"]=data.apply(lambda x:convertStringtoList(x,False),axis=1)

print("No of record with legth of POSList != WordList",data.apply(lambda x:len(x["POSList"])!=len(x["WordList"]),axis=1).sum())

data["TagToWordMap"]=data.apply(lambda x: WordToTagDic(x),axis=1)

data["wordCount"]=data.apply(lambda x: wordCount(x),axis=1)
notValidIndex=list(data[data["wordCount"]<=1].index) 
data=data.drop(notValidIndex)

data["UniqueWordCount"]=data.apply(lambda x: wordCount(x,True),axis=1)




No of record with legth of POSList != WordList 0


In [7]:
data.iloc[5]["Word"]

"['Mr.', 'Egeland', 'said', 'the', 'latest', 'figures', 'show', '1.8', 'million', 'people', 'are', 'in', 'need', 'of', 'food', 'assistance', '-', 'with', 'the', 'need', 'greatest', 'in', 'Indonesia', ',', 'Sri', 'Lanka', ',', 'the', 'Maldives', 'and', 'India', '.']"

In [8]:
data.iloc[5]["POS"]

"['NNP', 'NNP', 'VBD', 'DT', 'JJS', 'NNS', 'VBP', 'CD', 'CD', 'NNS', 'VBP', 'IN', 'NN', 'IN', 'NN', 'NN', ':', 'IN', 'DT', 'NN', 'JJS', 'IN', 'NNP', ',', 'NNP', 'NNP', ',', 'DT', 'NNP', 'CC', 'NNP', '.']"

In [9]:
data.iloc[5]["TagToWordMap"]

{'NNP': {'Mr.': 1,
  'Egeland': 1,
  'Indonesia': 1,
  'Sri': 1,
  'Lanka': 1,
  'Maldives': 1,
  'India': 1},
 'VBD': {'said': 1},
 'DT': {'the': 3},
 'JJS': {'latest': 1, 'greatest': 1},
 'NNS': {'figures': 1, 'people': 1},
 'VBP': {'show': 1, 'are': 1},
 'CD': {'1.8': 1, 'million': 1},
 'IN': {'in': 2, 'of': 1, 'with': 1},
 'NN': {'need': 2, 'food': 1, 'assistance': 1},
 'CC': {'and': 1}}

In [10]:
# TrainData,TestData=train_test_split(data, random_state=104,  test_size=0.01,  shuffle=True) 

In [11]:
TestData=data.sample(50,random_state=40)
TrainData=data.drop(TestData.index)


In [12]:
print("total data size:-",data.shape)
print("Train Data size",TrainData.shape)
print("Test Data size",TestData.shape)



total data size:- (46552, 10)
Train Data size (46502, 10)
Test Data size (50, 10)


In [13]:
def CreateMasterFreqDic(RowWiseMap):
    MasterCount={}
    for RowDic in RowWiseMap:
        for tags in RowDic:
            RowFreqCountDic=RowDic[tags]
            if tags in MasterCount:
                MasterFreqCountDic=MasterCount[tags]
                for word in RowFreqCountDic:
                    if word in MasterFreqCountDic:
                        MasterFreqCountDic[word]+=RowFreqCountDic[word]
                    else:
                        MasterFreqCountDic[word]=RowFreqCountDic[word]
                MasterCount[tags]=dict(MasterFreqCountDic)
            else:
                 MasterCount[tags]=dict(RowFreqCountDic)
    return MasterCount

            

In [14]:
FinalEmissionDic=CreateMasterFreqDic(TrainData["TagToWordMap"])


In [15]:
TotalTags=len(FinalEmissionDic.keys())
print("Total Tags in Train dataset:-",TotalTags)


Total Tags in Train dataset:- 36


In [16]:
count=0
for i in FinalEmissionDic.keys():
    count+=sum(FinalEmissionDic[i].values())
print("Total Words captured by FinalEmissionDic:-",count)
TotalWord=TrainData["wordCount"].sum()
print("Total words in the Train dataset:-",TotalWord)    
## Total words in dataset  should be equal to total words captured in our FinalEmissionDic

Total Words captured by FinalEmissionDic:- 930101
Total words in the Train dataset:- 930101


In [17]:
TrainData["wordCount"].mean()  ### average word length in the dataset 

np.float64(20.00131177153671)

In [18]:
### Dictionary for transitive probability 

In [19]:
def TransitiveMap(TagColumn):
    resultDict={}
    specialchar='[@_!#$%^&*()<>?/\|}{~:],.;:`` '

    for tags in TagColumn:
        taglist=list(tags)
        if "start" in resultDict:
            posCountDict=resultDict["start"]
            if taglist[0] in posCountDict:
                posCountDict[taglist[0]]+=1
            else:
                posCountDict[taglist[0]]=1
                
            resultDict["start"]=dict(posCountDict)
                
        else:
            resultDict["start"]={taglist[0]:1}
        for i in range(len(taglist)-1):
            
            if taglist[i] in specialchar:
                continue
                
            if taglist[i] in resultDict:
                posCountDict=resultDict[taglist[i]]
                if taglist[i+1] in posCountDict:
                    posCountDict[taglist[i+1]]+=1
                else:
                    posCountDict[taglist[i+1]]=1
                resultDict[taglist[i]]=dict(posCountDict)
            else:
                resultDict[taglist[i]]={taglist[i+1]:1}
                
    return resultDict

    

In [20]:
## calculate total number of uniqiue words which is our vocabulary 
x=0
UniqueWords=set()
for freqdict in FinalEmissionDic.values():
    for word in freqdict:
        UniqueWords.add(word)
        count+=1
vocab=len(UniqueWords)     

print("Unique number of words in the dataset is :-",vocab)

Unique number of words in the dataset is :- 34363


In [21]:
TransitiveDict=TransitiveMap(TrainData["POSList"])

In [22]:
sum(TransitiveDict["start"].values())

46502

In [23]:
TransitiveDict["CC"]

{'VB': 764,
 'JJ': 3415,
 'NNS': 1802,
 'NNP': 3303,
 'DT': 2615,
 'JJR': 95,
 'CD': 872,
 'NN': 2674,
 'VBD': 2028,
 'VBZ': 722,
 'VBP': 225,
 'PRP': 510,
 'RB': 660,
 'MD': 214,
 'TO': 150,
 'PRP$': 506,
 'VBG': 781,
 'VBN': 578,
 'IN': 852,
 'NNPS': 39,
 'WRB': 54,
 'RBS': 11,
 'EX': 47,
 'WP': 31,
 'RBR': 18,
 'JJS': 26,
 'FW': 1,
 'PDT': 2,
 'WDT': 4}

In [24]:
laplaceAlpha=0.000001
uniquesTags=list(FinalEmissionDic.keys())

In [25]:
sum(TransitiveDict["start"].values())

46502

In [26]:
## Emission probability with laplace smoothing   P(Thousands/nns)=
(FinalEmissionDic["NNS"]["Thousands"]+laplaceAlpha)/(sum(FinalEmissionDic["NNS"].values())+laplaceAlpha*vocab)


0.0015352343048914176

In [27]:
## Transitive probability  with laplace smoothing   p(nns nns) =
(TransitiveDict["NNS"]["NNS"]+laplaceAlpha)/(sum(TransitiveDict["NNS"].values())+laplaceAlpha*TotalTags)

0.014173128265059225

In [28]:
#### Training is complete Now Need to use Viterbi algorithm for prediction 

In [29]:
def veterbiRecursion(prevState,poslist,probTillNow,CurrIndex,sentence,UniqueWords):
    if probTillNow <=10**-7:
        return [poslist,probTillNow]
    if CurrIndex>=len(sentence):
        return [poslist,probTillNow]

    maxProb=0
    maxProbPath=[]
    check=0
    for tags in uniquesTags:
        if tags in TransitiveDict[prevState]:
            Pt=(TransitiveDict[prevState][tags]+laplaceAlpha)/(sum(TransitiveDict[prevState].values())+laplaceAlpha*TotalTags)
        else:
            Pt=0

        
        if sentence[CurrIndex] in FinalEmissionDic[tags]:
            Pe=(FinalEmissionDic[tags][sentence[CurrIndex]]+laplaceAlpha)/(sum(FinalEmissionDic[tags].values())+laplaceAlpha*vocab)
            finalprob=Pt*Pe*probTillNow*1000
            path,prob=veterbiRecursion(tags,poslist+[tags],finalprob,CurrIndex+1,sentence,UniqueWords)
            check=1
        else:
            # Pe=(laplaceAlpha)/(sum(FinalEmissionDic[tags].values())+laplaceAlpha*vocab)
            if sentence[CurrIndex] not in UniqueWords:
                Pe=0.0001
                finalprob=Pt*Pe*probTillNow*1000
                path,prob=veterbiRecursion(tags,poslist+["UK"],finalprob,CurrIndex+1,sentence,UniqueWords)
                check=1


        if check and prob>maxProb  :
            maxProbPath=list(path)
            maxProb=prob
    return [maxProbPath,maxProb]
            


In [30]:
len(TransitiveDict.keys())

37

In [31]:
FinalEmissionDic.keys()

dict_keys(['NNS', 'IN', 'VBP', 'VBN', 'NNP', 'TO', 'VB', 'DT', 'NN', 'CC', 'JJ', 'PRP', 'VBZ', 'VBG', 'VBD', 'WRB', 'JJR', 'RP', 'MD', 'RB', 'JJS', 'CD', 'POS', 'WDT', 'PRP$', 'WP', 'NNPS', 'WP$', 'EX', 'RBR', 'RBS', 'PDT', 'UH', 'LRB', 'RRB', 'FW'])

In [32]:
# sentence=['It', 'also', 'cited', 'voter', 'ignorance', 'of', 'the', 'nature', 'of', 'choice', 'among', 'the', 'candidates']

In [33]:
sentence=data.iloc[130]["Word"]

In [34]:
sentence=sentence[1:-1].replace("'","").replace(" ","").split(",")[0:-1]


In [35]:
print(sentence)

['It', 'also', 'cited', 'voter', 'ignorance', 'of', 'the', 'nature', 'of', 'choice', 'among', 'the', 'candidates']


In [36]:
data.iloc[130]["POS"]

"['PRP', 'RB', 'VBD', 'NN', 'NN', 'IN', 'DT', 'NN', 'IN', 'NN', 'IN', 'DT', 'NNS', '.']"

In [37]:
tag,prob=veterbiRecursion('start',[],1,0,sentence,UniqueWords)

In [38]:
print(len(tag))
len(sentence)

13


13

In [39]:
print(tag)  ### predicted Tags for the given sentence 

['PRP', 'RB', 'VBD', 'NN', 'NN', 'IN', 'DT', 'NN', 'IN', 'NN', 'IN', 'DT', 'NNS']


In [40]:
### Check for all the test sentence and calculate accuracy 

In [41]:
def predicttag(data):
    sentence=data["WordList"]
    actualTags=data["POSList"]

    tag,prob=veterbiRecursion('start',[],1,0,sentence,UniqueWords)
   
    return tag
    

In [42]:
TestData["predictedTag"]=TestData[["WordList","POSList"]].apply(lambda x:predicttag(x),axis=1)

In [43]:
TestData.head(3)

,Sentence_ID,Word,POS,Tag,validTagging,WordList,POSList,TagToWordMap,wordCount,UniqueWordCount,predictedTag
41662,Sentence: 47495,"['The', 'ministry', 'says', 'eight', 'armed', ...","['DT', 'NN', 'VBZ', 'CD', 'JJ', 'NNS', 'IN', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",True,"[The, ministry, says, eight, armed, assailants...","[DT, NN, VBZ, CD, JJ, NNS, IN, CD, NN, NNS, VB...","{'DT': {'The': 1, 'the': 1}, 'NN': {'ministry'...",14,14,"[DT, NN, VBZ, CD, JJ, NNS, IN, CD, NN, NNS, VB..."
26783,Sentence: 34102,"['A', 'FATHER', ',', 'being', 'on', 'the', 'po...","['DT', 'NN', ',', 'VBG', 'IN', 'DT', 'NN', 'IN...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",True,"[A, FATHER, being, on, the, point, of, death, ...","[DT, NN, VBG, IN, DT, NN, IN, NN, VBD, TO, VB,...","{'DT': {'A': 1, 'the': 2}, 'NN': {'FATHER': 1,...",29,26,"[DT, UK, VBG, IN, DT, NN, IN, NN, VBD, TO, VB,..."
40136,Sentence: 46120,"['French', 'Interior', 'Minister', 'Brice', 'H...","['JJ', 'NNP', 'NNP', 'NNP', 'NNP', 'VBZ', 'NNP...","['B-gpe', 'O', 'B-per', 'I-per', 'I-per', 'O',...",True,"[French, Interior, Minister, Brice, Hortefeux,...","[JJ, NNP, NNP, NNP, NNP, VBZ, NNP, NNP, VBZ, V...","{'JJ': {'French': 1, 'particular': 1}, 'NNP': ...",22,21,"[JJ, NNP, NNP, UK, NNP, VBZ, NNP, NNP, VBZ, VB..."


In [44]:
def CheckAccuracy(data):
    actualTags=data["POSList"]
    predictedTags=data["predictedTag"]

    count=0
    for i in range(len(predictedTags)):
        if predictedTags[i]==actualTags[i]:
            count+=1
    return count
    

In [45]:
TestData["CorrectPredictionCount"]=TestData.apply(lambda x:CheckAccuracy(x),axis=1)

In [46]:
TestData.head(3)

,Sentence_ID,Word,POS,Tag,validTagging,WordList,POSList,TagToWordMap,wordCount,UniqueWordCount,predictedTag,CorrectPredictionCount
41662,Sentence: 47495,"['The', 'ministry', 'says', 'eight', 'armed', ...","['DT', 'NN', 'VBZ', 'CD', 'JJ', 'NNS', 'IN', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",True,"[The, ministry, says, eight, armed, assailants...","[DT, NN, VBZ, CD, JJ, NNS, IN, CD, NN, NNS, VB...","{'DT': {'The': 1, 'the': 1}, 'NN': {'ministry'...",14,14,"[DT, NN, VBZ, CD, JJ, NNS, IN, CD, NN, NNS, VB...",14
26783,Sentence: 34102,"['A', 'FATHER', ',', 'being', 'on', 'the', 'po...","['DT', 'NN', ',', 'VBG', 'IN', 'DT', 'NN', 'IN...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",True,"[A, FATHER, being, on, the, point, of, death, ...","[DT, NN, VBG, IN, DT, NN, IN, NN, VBD, TO, VB,...","{'DT': {'A': 1, 'the': 2}, 'NN': {'FATHER': 1,...",29,26,"[DT, UK, VBG, IN, DT, NN, IN, NN, VBD, TO, VB,...",28
40136,Sentence: 46120,"['French', 'Interior', 'Minister', 'Brice', 'H...","['JJ', 'NNP', 'NNP', 'NNP', 'NNP', 'VBZ', 'NNP...","['B-gpe', 'O', 'B-per', 'I-per', 'I-per', 'O',...",True,"[French, Interior, Minister, Brice, Hortefeux,...","[JJ, NNP, NNP, NNP, NNP, VBZ, NNP, NNP, VBZ, V...","{'JJ': {'French': 1, 'particular': 1}, 'NNP': ...",22,21,"[JJ, NNP, NNP, UK, NNP, VBZ, NNP, NNP, VBZ, VB...",21


In [47]:
print("Total Word in test data:--",TestData["wordCount"].sum())
print("Total correct predicted Word  in test data:--",TestData["CorrectPredictionCount"].sum())

Total Word in test data:-- 977
Total correct predicted Word  in test data:-- 809


In [48]:
accuracy=TestData["CorrectPredictionCount"].sum()/TestData["wordCount"].sum()
print("accuracy:-",accuracy)

accuracy:- 0.8280450358239508
